# Open systems

In the previous demo, we looked at what happens when you switch from a periodic mesh to a rectangle mesh.
When you don't use any other boundary conditions, the equation builder assumes that you want rigid walls and as a consequence the waves reflect off of the walls rather than go back around to the other side.
In this demo, we'll continue on this theme but instead look at how to have an inflow and an outflow boundary.

In [ ]:
import firedrake
nx, ny = 24, 24
Lx, Ly = 20.0, 20.0
mesh = firedrake.RectangleMesh(nx, ny, Lx, Ly, diagonal='crossed')

In the last demo, every boundary was a rigid wall by default and we didn't have to figure out how to apply different conditions to different segments.
Here we'll need to actually know how to distinguish different segments because we'll want walls at the top and bottom, inflow at the left, and outflow at the right.
The mesh data structures in Firedrake come with a set of numerical boundary IDs.
We can visualize these using the Firedrake `triplot` function and adding a legend to the figure.

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
axes.set_aspect('equal')
firedrake.triplot(mesh, axes=axes)
axes.legend();

We'll once again use the DG(1)/BDFM(2) element pair.

In [ ]:
degree = 1
Q = firedrake.FunctionSpace(mesh, 'DG', degree)
V = firedrake.FunctionSpace(mesh, 'BDFM', degree + 1)

Z = Q * V
z_0 = firedrake.Function(Z)

Here we'll define the values of the thickness and momentum at the inflow boundary.
We'll use a very high inflow velocity -- 2.5 m/s -- in order to exaggerate the effect of a surface wave both spreading and being propagated downstream.

In [ ]:
from firedrake import inner, max_value, Constant

x = firedrake.SpatialCoordinate(mesh)

H = Constant(1.0)
u_in = Constant(2.5)
q_in = H * Constant((u_in, 0.0))

δh = 0.025
y = Constant((Lx / 2, Ly / 2))
r = Constant(Lx / 10)
h_expr = H + δh * max_value(0, 1 - inner(x - y, x - y) / r**2)

b = Constant(0.0) * x[0]

In [ ]:
h_0, q_0 = z_0.split()
h_0.project(h_expr)
q_0.project(q_in);

In [ ]:
import numpy as np
from plumes.coefficients import gravity
C = abs(float(u_in)) + np.sqrt(gravity * float(H))
δx = mesh.cell_sizes.dat.data_ro[:].min()
timestep = (δx / 4) / C / (2 * degree + 1)

final_time = 4 * Lx / C
num_steps = int(final_time / timestep)
dt = final_time / num_steps

output_time = 1 / 30
output_freq = max(int(output_time / dt), 1)

In [ ]:
from plumes.models import shallow_water
g = firedrake.Constant(gravity)
bcs = {
    'thickness_in': H,
    'momentum_in': q_in,
    'inflow_ids': (1,),
    'outflow_ids': (2,)
}
equation = shallow_water.make_equation(g, b, **bcs)

In [ ]:
import tqdm
from plumes import numerics
integrator = numerics.SSPRK34(equation, z_0)

hs = []
qs = []

progress_bar = tqdm.trange(num_steps)
for step in progress_bar:
    if step % output_freq == 0:
        z = integrator.state
        h, q = z.split()
        hmin, hmax = h.dat.data_ro[:].min(), h.dat.data_ro[:].max()
        progress_bar.set_description(f'{hmin:5.3f}, {hmax:5.3f}')
        hs.append(h.copy(deepcopy=True))
        qs.append(q.copy(deepcopy=True))
    
    integrator.step(dt)

In [ ]:
%%capture
Q0 = firedrake.FunctionSpace(mesh, 'DG', 0)
η = firedrake.project(hs[0] + b, Q0)

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

fig, axes = plt.subplots()
axes.set_aspect('equal')
axes.get_xaxis().set_visible(False)
axes.get_yaxis().set_visible(False)
axes.set_xlim((0, Lx))
axes.set_ylim((0, Ly))
colors = firedrake.tripcolor(
    η, num_sample_points=1, vmin=0.95, vmax=1.05, axes=axes
)

def animate(h):
    η.project(h + b)
    colors.set_array(η.dat.data_ro[:])

interval = 1e3 * output_freq * dt
animation = FuncAnimation(fig, animate, frames=hs, interval=interval)

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())